In [1]:
import pandas as pd
import statsmodels.formula.api as sm
df = pd.DataFrame.from_csv("StockData.csv", encoding="UTF-8")
model = sm.ols(formula="PXD ~ Index + Oil + Gold + NaturalGas", data=df)

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
result = model.fit()

In [3]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    PXD   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                     452.7
Date:                Sun, 05 Nov 2017   Prob (F-statistic):          8.10e-267
Time:                        13:22:27   Log-Likelihood:                 4624.4
No. Observations:                1728   AIC:                            -9239.
Df Residuals:                    1723   BIC:                            -9211.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0004      0.000      0.976      0.3

In [4]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    print(result.summary())
regress("F")

                            OLS Regression Results                            
Dep. Variable:                      F   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     389.8
Date:                Sun, 05 Nov 2017   Prob (F-statistic):          3.11e-239
Time:                        13:22:27   Log-Likelihood:                 5022.8
No. Observations:                1728   AIC:                        -1.004e+04
Df Residuals:                    1723   BIC:                        -1.001e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0003      0.000     -0.981      0.3

In [5]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    print(result.pvalues)
regress("AAPL")

Intercept      2.277948e-01
Index         1.951953e-132
Oil            1.551144e-02
Gold           7.300774e-02
NaturalGas     4.798328e-03
dtype: float64


In [6]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    print(result.pvalues<.05)
regress("PXD")

Intercept     False
Index          True
Oil            True
Gold          False
NaturalGas    False
dtype: bool


In [7]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    print(result.pvalues<.05)
    print(result.params)
regress("PXD")

Intercept     False
Index          True
Oil            True
Gold          False
NaturalGas    False
dtype: bool
Intercept     0.000392
Index         1.228180
Oil           0.384341
Gold          0.050847
NaturalGas    0.005419
dtype: float64


In [8]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    truthSeries = result.pvalues<.05
    print(result.params[truthSeries])
regress("PXD")

Index    1.228180
Oil      0.384341
dtype: float64


In [9]:
def regress(stock):
    result = sm.ols(formula=stock+" ~ Index + Oil + Gold + NaturalGas", data=df).fit()
    return pd.DataFrame(result.params[result.pvalues<.05],columns=[stock])
regress("PXD")

,PXD
Index,1.228180
Oil,0.384341


In [10]:
regress("BRK.B")

PatsyError: Error evaluating factor: NameError: name 'BRK' is not defined
    BRK.B ~ Index + Oil + Gold + NaturalGas
    ^^^^^

In [11]:
cols = []
for x in df.columns:
    if "." in x:
        x = x.replace(".","")
    if ":" in x:
        x = x.replace(":","")
    cols.append(x)
df.columns = cols

In [12]:
df2 = pd.DataFrame()
for stock in df.columns[:-4]:
    frame = regress(stock)
    df2 = pd.concat([df2,frame], axis=1)
print(df2)

                   A       AAL       AAP      AAPL      ABBV       ABC  \
Gold             NaN       NaN       NaN       NaN       NaN       NaN   
Index       1.444761  1.690434  0.809428  0.980275  1.134925  0.760971   
Intercept        NaN       NaN       NaN       NaN       NaN       NaN   
NaturalGas       NaN       NaN       NaN  0.025109       NaN       NaN   
Oil              NaN -0.303885 -0.059674 -0.039810       NaN -0.051730   

                 ABT       ACN      ADBE       ADI    ...          XL  \
Gold             NaN       NaN       NaN       NaN    ...         NaN   
Index       0.777777  1.001813  1.176556  1.169728    ...     1.14801   
Intercept        NaN       NaN       NaN       NaN    ...         NaN   
NaturalGas       NaN       NaN       NaN       NaN    ...         NaN   
Oil        -0.034620       NaN       NaN       NaN    ...         NaN   

                XLNX       XOM      XRAY       XRX       XYL       YUM  \
Gold             NaN       NaN       NaN -

In [13]:
df2.to_csv("RegressionMatrix1.csv",encoding="UTF-8")